# Auxiliary Scripts

## Database Loader

In [2]:
#python script that read files and then insert the data into the database
#created by Raphael Duarte

In [3]:
#import sqlalchemy
import sqlalchemy as sqa
from sqlalchemy.orm import declarative_base
from sqlalchemy import func
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship


import timeit #for timing
from datetime import datetime #for the timestamp
import numpy as np

from sqlalchemy.orm import sessionmaker

C:\Users\Raphael\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Raphael\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Raphael\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
sqa.__version__  

'1.4.45'

In [5]:
#creates engine to the database file
engine = sqa.create_engine("sqlite:///Data/movie.db")

In [6]:
#creates declarative base
Base = declarative_base()

In [7]:
#creates class for communicating with the movie table
class Movies(Base):
    __tablename__ = 'movies'
    
    id_movie = sqa.Column(sqa.Integer, primary_key=True)
    title_movie = sqa.Column(sqa.String)
    year_movie = sqa.Column(sqa.Integer)
    
    
    def __repr__(self):
        return "<Movie(id_movie='%i',title_movie='%s')>" % (self.id_movie, self.title_movie)

In [8]:
#creates class for communicating with the movie table
class Ratings(Base):
    __tablename__ = 'ratings'
    
    id_rating = sqa.Column(sqa.Integer, primary_key=True)
    id_movie = sqa.Column(sqa.Integer)
    id_user = sqa.Column(sqa.Integer)
    value_rating = sqa.Column(sqa.Float)
    timestamp_rating = sqa.Column(sqa.String)

    
    def __repr__(self):
        return "<Rating(id_rating='%i',id_movie='%i',id_user='%i',value_rating='%i',timestamp_rating='%s')>" % (self.id_rating,self.id_movie,self.id_user,self.value_rating, self.timestamp_rating)

In [9]:
#creates class for communicating with the movie table
class GenreList(Base):
    __tablename__ = 'genre_list'
    
    id_genre_item = sqa.Column(sqa.Integer, primary_key=True)
    id_movie = sqa.Column(sqa.Integer)
    name_genre = sqa.Column(sqa.String)
    
    
    def __repr__(self):
        return "<GenreList(id_genre_item='%i',id_movie='%i',name_genre='%s')>" % (self.id_genre_item,self.id_movie,self.id_user,self.name_genre)

In [10]:
#creates class for communicating with the movie table
class ReferenceRatings(Base):
    __tablename__ = 'reference_ratings'
    
    id_reference = sqa.Column(sqa.Integer, primary_key=True)
    average_rating = sqa.Column(sqa.Float)
    timestamp_update = sqa.Column(sqa.String)
    onestar_rating = sqa.Column(sqa.Integer)
    twostar_rating = sqa.Column(sqa.Integer)
    threestar_rating = sqa.Column(sqa.Integer)
    fourstar_rating = sqa.Column(sqa.Integer)
    fivestar_rating = sqa.Column(sqa.Integer)
    count_rating = sqa.Column(sqa.Integer)

    
    def __repr__(self):
        return "<Rating(id_movie='%i',average_rating='%i',timestamp_update='%s')>" % (self.id_movie,self.average_rating, self.timestamp_update)

In [11]:
class intersec_movies(Base):
    __tablename__ = 'intersec_movies'
    
    id_intersec = sqa.Column(sqa.Integer, primary_key=True, index=True)
    id_movieA = sqa.Column(sqa.Integer)
    id_movieB = sqa.Column(sqa.Integer)
    count_value = sqa.Column(sqa.Integer)
    
    def __repr__(self):
        return "<intersec_movies(id_moviAe='%i',id_moviAe='%i',count='%i')>" % (self.id_movieA,self.id_movieB, self.count)

In [10]:
Movies.__table__

Table('movies', MetaData(), Column('id_movie', Integer(), table=<movies>, primary_key=True, nullable=False), Column('title_movie', String(), table=<movies>), Column('year_movie', Integer(), table=<movies>), schema=None)

In [11]:
ReferenceRatings.__table__

Table('reference_ratings', MetaData(), Column('id_reference', Integer(), table=<reference_ratings>, primary_key=True, nullable=False), Column('average_rating', Float(), table=<reference_ratings>), Column('timestamp_update', String(), table=<reference_ratings>), Column('onestar_rating', Integer(), table=<reference_ratings>), Column('twostar_rating', Integer(), table=<reference_ratings>), Column('threestar_rating', Integer(), table=<reference_ratings>), Column('fourstar_rating', Integer(), table=<reference_ratings>), Column('fivestar_rating', Integer(), table=<reference_ratings>), Column('count_rating', Integer(), table=<reference_ratings>), schema=None)

In [12]:
#connects to the database
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
#clear the movie table
#mvs = session.query(Movies) #queries references
 
#mvs.delete() #delete reference

#session.commit()

In [ ]:
session.rollback()

In [ ]:
session.flush()

In [ ]:
#load the movies data into the movies table

#open the file

with open("Data/ml-25m/movies.csv", encoding='utf-8') as f:
    f.readline() #skip first line
    count=1
    for line in f:
        items = line.split(sep=',')
        movieid = items[0]
        #process the title and year
        title = (",".join(items[1:-1])).strip('\"').strip() #join all different items inside the title
        print(title)
        if(title[-6:][1:-1].isnumeric()): #check if year is correct
            year = int(title[-6:][1:-1]) #gets the year from the title
            title = title[:-7] #removes the year from the title if it is ther
        else:
            year = -1 #sets as -1 for unknown year
        
        #creates object
        mv = Movies(id_movie=int(movieid), title_movie=title, year_movie=year)
        #adds to section
        session.add(mv)
        genres = items[-1].split(sep="|")
        for genre in genres:
            genre = genre.strip('\n').strip('\n')
            gl = GenreList(id_genre_item=count,id_movie=int(movieid),name_genre=genre)
            count+=1
            #session.add(gl)
        if(count%10000 == 0): 
            print(count)
            session.commit()
    session.commit()
        
    

In [ ]:
#load the ratings data into the rating table

#open the file

with open("Data/ml-25m/ratings.csv", encoding='utf-8') as f:
    f.readline() #skip first line
    count=1
    for line in f:
        items = line.split(sep=',')
        userid = items[0]
        movieid = items[1]
        rating = items[2]
        timestamp = items[3]
        #print(movieid)
        rt = Ratings(id_rating=count, id_movie=int(movieid), id_user=int(userid), value_rating=float(rating), timestamp_rating=timestamp)
        count+=1
        #session.add(rt)
        if(count%1000000 == 0): 
            print(count)
            #session.commit()
    #session.commit()

## Database Updater

In [13]:
max_movie = session.query(func.max(Movies.id_movie)).all()[0][0] + 1 #number of movies

aavg = np.zeros(max_movie)
acount = np.zeros(max_movie)
aone = np.zeros(max_movie)
atwo = np.zeros(max_movie)
athree = np.zeros(max_movie)
afour = np.zeros(max_movie)
afive = np.zeros(max_movie)

#queries for temporary information
one_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 1).group_by(Ratings.id_movie).order_by(Ratings.id_movie)
two_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 2, Ratings.value_rating > 1).group_by(Ratings.id_movie).order_by(Ratings.id_movie) 
three_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 3, Ratings.value_rating > 2).group_by(Ratings.id_movie).order_by(Ratings.id_movie)
four_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 4, Ratings.value_rating > 3).group_by(Ratings.id_movie).order_by(Ratings.id_movie)
five_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 5, Ratings.value_rating > 4).group_by(Ratings.id_movie).order_by(Ratings.id_movie)
average = session.query(Ratings.id_movie, func.avg(Ratings.value_rating), func.count(Ratings.value_rating)).group_by(Ratings.id_movie).order_by(Ratings.id_movie)


list_values = zip(average,one_star,two_star,three_star,four_star,five_star)

for item in list_values:
    aavg[item[0][0]] = item[0][1]
    acount[item[0][0]] = item[0][2]
    aone[item[1][0]] = item[1][1]
    atwo[item[2][0]] = item[2][1]
    athree[item[3][0]] = item[3][1]
    afour[item[4][0]] = item[4][1]
    afive[item[5][0]] = item[5][1]

In [14]:
rfavg = session.query(ReferenceRatings) #queries references
 
rfavg.delete() #delete reference

session.commit()

In [15]:

rfavg = session.query(ReferenceRatings) #queries references
 
rfavg.delete() #delete reference

for i in range(0, max_movie):
    idm = i
    avg = aavg[i]
    time = datetime.now().strftime("%Y/%m/%d, %H:%M:%S")
    one = aone[i]
    two = atwo[i]
    three = athree[i]
    four = afour[i]
    five = afive[i]
    count = acount[i]
    
    
    rf = ReferenceRatings(id_reference=idm, 
                          average_rating=avg, 
                          timestamp_update=time, 
                          onestar_rating=one, 
                          twostar_rating=two,
                          threestar_rating=three,
                          fourstar_rating=four,
                          fivestar_rating=five,
                          count_rating=count)
    session.add(rf)
session.commit()
    